# Imports and Libraries

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

DATA_DIR = Path("./Data")
FLIGHTS_PATH = DATA_DIR / "Flight_on_time_HIX_CA.csv"
WEATHER_HIX_PATH = DATA_DIR / "weather_cleaned_HIX.csv"

In [2]:
w = pd.read_csv(WEATHER_HIX_PATH)
f = pd.read_csv(FLIGHTS_PATH)

/var/folders/h5/hngl5mzn2g1gqq64g9h1wz9w0000gn/T/ipykernel_34431/2936072061.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  f[sched_dep_col] = pd.to_datetime(f[sched_dep_col], errors="coerce", utc=False)


,airline,flight_number,plane_id,flightdate,origin_airport,destination_airport,flight_distance,scheduled_departure_time,actual_departure_time,departure_delay_minutes,...,arr_hour,is_redeye,departure_delay_recomputed,arrival_delay_recomputed,scheduled_blocktime_min,actual_blocktime_min,total_taxi_time_min,airborne_time_min,is_departure_delayed_15,is_arrival_delayed_15
0,CA,6752,GBHWFM,2019-12-01,HIX,VYM,760.0,2025-10-26 00:15:00,00:14:00,0.0,...,3,1,0.0,2.0,198.0,201.0,32.0,169.0,0,0
174,CA,7217,GIIUNG,2019-12-02,HIX,TMJ,632.0,2025-10-26 00:15:00,00:11:00,0.0,...,3,1,0.0,27.0,175.0,206.0,48.0,158.0,0,1
175,CA,7240,GHLWFY,2019-12-02,HIX,FTC,403.0,2025-10-26 00:15:00,02:14:00,119.0,...,1,1,119.0,114.0,94.0,89.0,29.0,60.0,1,1


In [6]:
# ensure sorting by join key
w_sorted = w.sort_values(w.columns[1])  # time_col is 2nd because base_cols = [airport_col, time_col]
f_sorted = f.sort_values('scheduled_departure_time')

# pandas merge_asof needs explicit column names
weather_time = w.columns[1]

# join with a reasonable tolerance (e.g., 3 hours)
merged = pd.merge_asof(
    f_sorted,
    w_sorted,
    left_on='scheduled_departure_time',
    right_on=weather_time,
    by=None,            # origin airport is HIX for all rows post-filtering
    direction="backward",
    tolerance=pd.Timedelta("3H")
)

# flag rows where weather not found within tolerance
merged["weather_missing"] = merged[weather_time].isna()

merged.head(3)

/var/folders/h5/hngl5mzn2g1gqq64g9h1wz9w0000gn/T/ipykernel_34431/2675041256.py:16: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  tolerance=pd.Timedelta("3H")


MergeError: Incompatible merge dtype, dtype('O') and dtype('<M8[ns]'), both sides must have numeric dtype